## Ограничения на ресурсы для numpy, импорт библиотек

In [2]:
import os
os.environ["OMP_NUM_THREADS"] = "4" # export OMP_NUM_THREADS=4
os.environ["OPENBLAS_NUM_THREADS"] = "4" # export OPENBLAS_NUM_THREADS=4 
os.environ["MKL_NUM_THREADS"] = "6" # export MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = "4" # export VECLIB_MAXIMUM_THREADS=4
os.environ["NUMEXPR_NUM_THREADS"] = "6" # export NUMEXPR_NUM_THREADS=6

In [3]:
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
from functools import reduce

import scipy
from scipy.sparse import csr_matrix, hstack, vstack
import gzip
import pickle

import sklearn
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import QuantileTransformer, Normalizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, f1_score

from multiprocessing.pool import Pool
import re

import tensorflow as tf

from hyperopt import hp, fmin, tpe, atpe, Trials, STATUS_OK
import time
import warnings
import os
import gzip
from sklearn.exceptions import ConvergenceWarning

from IPython.display import clear_output
from functools import partial

warnings.filterwarnings("ignore", category=ConvergenceWarning)

## Загрузка мешка слов по регионам

In [5]:
with gzip.open('files/region_name_cbag_v2.pickle.gz', 'rb') as f:
    regmap = pickle.load(f)

reg_id = np.array(regmap['data'].argmax(axis=1)).flatten()
pd.Series(
    reg_id
).value_counts().head(50).index

_id_label, _id_cnt = np.unique(reg_id, return_counts=True)
_id_map = {}
regionmap = csr_matrix(regmap['data'][:, _id_label[_id_cnt>20]])
print(regionmap.shape)
for cid in _id_label[_id_cnt>20]:
    _id_map[cid] = len(_id_map)
reg_id = np.array([_id_map.get(cid, len(_id_map)) for cid in reg_id])
del _id_label, _id_cnt, _id_map
reg_id.max()

(415317, 79)


79

## Загрузка мешка слов по городам

In [6]:
with gzip.open('files/city_name_cbag_v2.pickle.gz', 'rb') as f:
    regmap = pickle.load(f)

city_id = np.array(regmap['data'].argmax(axis=1)).flatten()
pd.Series(
    city_id
).value_counts().head(50).index

_id_label, _id_cnt = np.unique(city_id, return_counts=True)
_id_map = {}
citmap = csr_matrix(regmap['data'][:, _id_label[_id_cnt>20]])
print(citmap.shape)
for cid in _id_label[_id_cnt>20]:
    _id_map[cid] = len(_id_map)
city_id = np.array([_id_map.get(cid, len(_id_map)) for cid in city_id])
del _id_label, _id_cnt, _id_map
city_id.max()

(415317, 661)


661

## Загрузка мешка слов по производителю

In [7]:
with gzip.open('files/cpe_manufacturer_name_cbag_v2.pickle.gz', 'rb') as f:
    regmap = pickle.load(f)

cpeman_id = np.array(regmap['data'].argmax(axis=1)).flatten()
pd.Series(
    cpeman_id
).value_counts().head(50).index

_id_label, _id_cnt = np.unique(cpeman_id, return_counts=True)
_id_map = {}
cpemanmap = csr_matrix(regmap['data'][:, _id_label[_id_cnt>20]])
print(cpemanmap.shape)
for cid in _id_label[_id_cnt>20]:
    _id_map[cid] = len(_id_map)
cpeman_id = np.array([_id_map.get(cid, len(_id_map)) for cid in cpeman_id])
del _id_label, _id_cnt, _id_map
cpeman_id.max()

(415317, 27)


27

## Загрузка мешка слов по устройству

In [8]:
with gzip.open('files/cpe_model_name_cbag_v2.pickle.gz', 'rb') as f:
    regmap = pickle.load(f)

cpemodname_id = np.array(regmap['data'].argmax(axis=1)).flatten()
pd.Series(
    cpemodname_id
).value_counts().head(50).index

_id_label, _id_cnt = np.unique(cpemodname_id, return_counts=True)
_id_map = {}
cpemodnamemap = csr_matrix(regmap['data'][:, _id_label[_id_cnt>20]])
print(cpemodnamemap.shape)
for cid in _id_label[_id_cnt>20]:
    _id_map[cid] = len(_id_map)
cpemodname_id = np.array([_id_map.get(cid, len(_id_map)) for cid in cpemodname_id])
del _id_label, _id_cnt, _id_map
cpemodname_id.max()

(415317, 396)


396

## Загрузка мешка слов по cpe_type_cd

In [9]:
with gzip.open('files/cpe_type_cd_cbag_v2.pickle.gz', 'rb') as f:
    regmap = pickle.load(f)

cpetype_id = np.array(regmap['data'].argmax(axis=1)).flatten()
cpetypemap = csr_matrix(regmap['data'])
print(cpetypemap.shape)
pd.Series(
    cpetype_id
).value_counts().head(50).index

(415317, 5)


Int64Index([0, 2, 1, 3], dtype='int64')

## Загрузка мешка слов по датам

In [10]:
with gzip.open('files/date_cbag_v2.pickle.gz', 'rb') as f:
    regmap = pickle.load(f)

date_id = np.array(regmap['data'].argmax(axis=1)).flatten()
datemap = csr_matrix(regmap['data'])
print(datemap.shape)
pd.Series(
    date_id
).value_counts().tail(50)

_id_label, _id_cnt = np.unique(date_id, return_counts=True)
_id_map = {}
datemap = csr_matrix(regmap['data'][:, _id_label[_id_cnt>20]])
print(datemap.shape)
for cid in _id_label[_id_cnt>20]:
    _id_map[cid] = len(_id_map)
date_id = np.array([_id_map.get(cid, len(_id_map)) for cid in date_id])
del _id_label, _id_cnt, _id_map
date_id.max()

(415317, 397)
(415317, 203)


203

## Загрузка мешка слов по времени суток

In [11]:
with gzip.open('files/part_of_day_cbag_v2.pickle.gz', 'rb') as f:
    regmap = pickle.load(f)

pod_id = np.array(regmap['data'].argmax(axis=1)).flatten()
podmap = csr_matrix(regmap['data'])
print(podmap.shape)
pd.Series(
    pod_id
).value_counts().tail(50)

(415317, 5)


1    217543
2    126023
0     64969
3      6782
dtype: int64

## Загрузка мешка слов по ценам

In [12]:
with gzip.open('files/price_cbag_v2.pickle.gz', 'rb') as f:
    regmap = pickle.load(f)

prices = np.array(pd.read_csv('files/price_mapper.tsv.gz', sep='\t').price.fillna(20_000).tolist()
                  + [20_000])
price_id = regmap['data']
price_id = (price_id.dot(prices)/np.array(price_id.sum(axis=1)).flatten())
pricemap_id = regmap['data'].dot(KBinsDiscretizer(n_bins=31,
                                                  strategy='kmeans',).fit_transform(prices[:, None]**0.5))

## Загрузка мешка слов по ссылкам

In [13]:
with gzip.open('files/url_host_cbag_v2.pickle.gz', 'rb') as f:
    datamap = pickle.load(f)
datamap

{'data': <415317x199684 sparse matrix of type '<class 'numpy.uint32'>'
 	with 32277669 stored elements in Compressed Sparse Row format>,
 'uids': array([     4,     16,     18, ..., 415276, 415288, 415293])}

## Загрузка признаков тайтлов ссылок

In [14]:
with gzip.open('auxilary/domain20k_title.pickle.gz', 'rb') as f:
    titlemap = pickle.load(f)

map_df = pd.read_csv('auxilary/url_host_mapper_v2.tsv.gz', sep='\t')

titlemap = \
datamap['data'][:, pd.DataFrame(
    dict(url_host=titlemap['domain'])
        ).merge(map_df).url_host_idx.values
               ].dot(CountVectorizer(ngram_range=(1,2),
                                     min_df=2,).\
                     fit_transform(titlemap['title']))

titlemap = csr_matrix((np.log2(1+titlemap.data), titlemap.nonzero()),
                      shape=titlemap.shape, dtype=np.float32)
titlemap

<415317x13118 sparse matrix of type '<class 'numpy.float32'>'
	with 82784390 stored elements in Compressed Sparse Row format>

## Загрузка эмбеддингов скриншотов

In [15]:
with gzip.open('auxilary/clipVIT_scores_20k.pickle.gz', 'rb') as f:
    clipmap = pickle.load(f)

clipmap = \
TfidfTransformer(sublinear_tf=True, norm=None).\
    fit_transform(
        datamap['data'][:, pd.DataFrame(
            dict(url_host=clipmap['domains'])
                ).merge(map_df).url_host_idx.values
                       ]
    ).\
    dot(
        np.array(clipmap['scores'])
    )

clipmap = Normalizer().fit_transform(clipmap)
clipmap = np.float32(clipmap)

clipmap.shape

(415317, 768)

## Загрузка признаков Doc2Vec

In [16]:
with gzip.open('doc2vec_feats_128x4.pickle.gz', 'rb') as f:
    doc2vec_feats = pickle.load(f)

## Загрузка признаков HTML

In [17]:
%%time

if not os.path.exists('html_feats.pickle.gz'):


    with gzip.open('auxilary/domain20k_html.txt.gz', 'rt',
                   encoding='utf-8') as f:
        html_data = f.read().split('\n=\n=\n')
        print(len(html_data))

    def tokenize(x):
        return x[0], re.findall('(?u)\\b\\w\\w+\\b', x[-1].lower())

    with Pool(20) as pool:
        pool_pbar = tqdm()
        html_tokens = [None for _ in range(len(html_data[:-1]))]
        for i, r in pool.imap(tokenize, enumerate(html_data[:-1])):
            html_tokens[i] = r
            pool_pbar.update(1)

    html_cbag = \
    CountVectorizer(ngram_range=(1,2),
                    min_df=300,
                    lowercase=False,
                    tokenizer=lambda x: x,
                    max_df=0.25).\
                   fit_transform(html_tokens)
    print(html_cbag.shape)

    html_tfidf = TfidfTransformer(sublinear_tf=True).fit_transform(html_cbag)

    from sklearn.decomposition import TruncatedSVD
    html_svd = TruncatedSVD(n_components=256,
                            random_state=10,
                            n_iter=3,).fit_transform(html_tfidf)
    print(html_svd.shape)

    with gzip.open('auxilary/domain20k_title.pickle.gz', 'rb') as f:
        htmlmap = pickle.load(f)
    htmlmap = \
    TfidfTransformer(sublinear_tf=True, norm=None).\
        fit_transform(
            datamap['data'][:, pd.DataFrame(
                dict(url_host=htmlmap['domain'])
                    ).merge(map_df).url_host_idx.values
                           ]
        ).\
        dot(
            html_svd
        )

    htmlmap = Normalizer().fit_transform(htmlmap)
    htmlmap = np.float32(htmlmap)

    print(htmlmap.shape)

    with gzip.open('auxilary/html_feats.pickle.gz', 'wb') as f:
        pickle.dump(htmlmap, f, protocol=-1)

with gzip.open('auxilary/html_feats.pickle.gz', 'rb') as f:
    htmlmap = pickle.load(f)

CPU times: user 3.02 s, sys: 646 ms, total: 3.66 s
Wall time: 3.67 s


## Загрузка признаков SimilarWeb

In [18]:
with gzip.open('auxilary/simweb_domain.pickle.gz', 'rb') as f:
    simweb_feats = pickle.load(f)

## Загрузка признаков Bigram ссылок

In [19]:
with gzip.open('auxilary/bigrams_dense.pickle.gz', 'rb') as f:
    bigrams_feats = pickle.load(f)

## Получение частотных ссылок мешка слов url_host

In [20]:
feats_mask = (np.array((datamap['data']>0).sum(axis=0)).flatten() > 40)
feats_mask.sum()

20144

## Считывание файла с таргетами

In [21]:
trg_df = pd.read_csv('target.tsv.gz', sep='\t')
trg_df.sample(10)

,user_id,age,is_male
163642,389301,23.0,1.0
208957,11227,42.0,0.0
326661,356595,41.0,0.0
339384,66835,20.0,1.0
372068,395895,42.0,1.0
367227,346685,NaN,NaN
392838,189980,41.0,1.0
85509,23586,NaN,NaN
281517,320627,NaN,NaN
398586,248151,NaN,NaN


# Подготовка таргетов и поднабора юзеров из обучения

In [22]:
key = 'data'

(trg_df.age.isna()|trg_df.is_male.isna()).sum(),\

all_mask = (~trg_df.age.isna()|~trg_df.is_male.isna()).values.copy()
trg_train = trg_df[all_mask].fillna({'is_male': 0.5, 'age': 34})
trg_age = trg_train.age.values.copy()
trg_sex = trg_train.is_male.values.copy()

X_tr = datamap[key][all_mask][:, feats_mask]

age_bins = [[0, 25], [26, 35], [36, 45], [46, 55], [56, 65], [66, 999]]

print('Train sample:', all_mask.sum())

y_all = 0
for k, age_bin in enumerate(age_bins):
    y = pd.Series(trg_age).between(*age_bin).values.copy()
    y_all += y*(k+1)
y_all.min(), y_all.max(), X_tr.shape

Train sample: 270000


(1, 6, (270000, 20144))

# Объединение мешков слов второстепенных доменов в один

In [23]:
cbag_all = hstack([regionmap,
                   citmap,
                   cpemanmap,
                   cpemodnamemap,
                   cpetypemap,
                   podmap,
                   datemap,
                   pricemap_id
                  ])
cbag_all = QuantileTransformer(n_quantiles=10).fit_transform(cbag_all)
cbag_all = csr_matrix(cbag_all)
cbag_all_train = cbag_all[all_mask]
cbag_all_train.shape

(270000, 1407)

# Переобозначения обучающих поднаборов признаков ради удобства

In [24]:
doc2vec_feats_train = doc2vec_feats[all_mask].copy()
print(doc2vec_feats_train.shape)
clipmap_train = clipmap[all_mask].copy()
print(clipmap_train.shape)
titlemap = csr_matrix(titlemap, dtype=np.float32)
titlemap_train = titlemap[all_mask]
print(titlemap_train.shape)
htmlmap_train = htmlmap[all_mask].copy()
print(htmlmap_train.shape)
simweb_feats_train = simweb_feats[all_mask].copy()
print(simweb_feats_train.shape)
bigrams_feats_train = bigrams_feats[all_mask].copy()
print(bigrams_feats_train.shape)

(270000, 512)
(270000, 768)
(270000, 13118)
(270000, 256)
(270000, 199)
(270000, 512)


# Подготовка мешка слов ссылок

In [25]:
all_data = datamap[key][:, feats_mask]
all_data_sqrt = csr_matrix((all_data.data**0.5, all_data.nonzero()),
                     shape=all_data.shape,
                     dtype=np.float32)
del all_data

# Настройка валидации

In [26]:
kfold = StratifiedKFold(n_splits=10,
                        shuffle=True,
                        random_state=101010)
folds = [(train_ind, test_ind) for train_ind, test_ind in
         kfold.split((np.uint8(trg_sex*2)+y_all*10).astype(str),
                     (np.uint8(trg_sex*2)+y_all*10).astype(str))]

## Функция с архитектурой модели (последняя версия)

In [32]:
def get_model_all(emb_size=8, dense_size=512, l1_reg=1e-7,
                  base_lr=2e-4, sex_weight=1, age_weight=1,
                  **kwargs):
    l1reg = tf.keras.regularizers.l1(l1_reg)

    # urls
    inp = tf.keras.layers.Input((X_tr.shape[1],), sparse=False)
    x = inp
    x = tf.keras.layers.Dense(dense_size, activation='relu',
                              use_bias=False,
                              kernel_regularizer=l1reg)(x)

    use_feats = kwargs['use_feats']
    activation = kwargs.get('use_emb_act', 'linear')

    # region id
    inp2 = tf.keras.layers.Input((1,), sparse=False)
    x2 = tf.keras.layers.Embedding(reg_id.max()+1,
                                   kwargs.get('e_region', emb_size),
                                   embeddings_regularizer=l1reg)(inp2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation(activation)(x2)

    # city id
    inp3 = tf.keras.layers.Input((1,), sparse=False)
    x3 = tf.keras.layers.Embedding(city_id.max()+1,
                                   kwargs.get('e_city', emb_size),
                                   embeddings_regularizer=l1reg)(inp3)
    x3 = tf.keras.layers.Flatten()(x3)
    x3 = tf.keras.layers.Activation(activation)(x3)

    # cpeman id
    inp4 = tf.keras.layers.Input((1,), sparse=False)
    x4 = tf.keras.layers.Embedding(cpeman_id.max()+1,
                                   kwargs.get('e_cpeman', emb_size),
                                   embeddings_regularizer=l1reg)(inp4)
    x4 = tf.keras.layers.Flatten()(x4)
    x4 = tf.keras.layers.Activation(activation)(x4)

    # cpemodname id
    inp5 = tf.keras.layers.Input((1,), sparse=False)
    x5 = tf.keras.layers.Embedding(cpemodname_id.max()+1,
                                   kwargs.get('e_cpemodname', emb_size),
                                   embeddings_regularizer=l1reg)(inp5)
    x5 = tf.keras.layers.Flatten()(x5)
    x5 = tf.keras.layers.Activation(activation)(x5)

    # cpetype_id id
    inp6 = tf.keras.layers.Input((1,), sparse=False)
    x6 = tf.keras.layers.Embedding(cpetype_id.max()+1,
                                   kwargs.get('e_cpetype', emb_size),
                                   embeddings_regularizer=l1reg)(inp6)
    x6 = tf.keras.layers.Flatten()(x6)
    x6 = tf.keras.layers.Activation(activation)(x6)

    # price id
    inp7 = tf.keras.layers.Input((1,), sparse=False)
    x7 = tf.keras.layers.Dense(kwargs.get('e_price', emb_size),
                               activation='tanh',
                               kernel_regularizer=l1reg)(inp7)

    # cbagmap
    inp8 = tf.keras.layers.Input((cbag_all.shape[-1],), sparse=False)
    x8 = tf.keras.layers.Dense(kwargs.get('e_cbag', emb_size*2),
                               activation='relu',
                               kernel_regularizer=l1reg)(inp8)

    # date_id id
    inp9 = tf.keras.layers.Input((1,), sparse=False)
    x9 = tf.keras.layers.Embedding(date_id.max()+1,
                                   kwargs.get('e_date', emb_size),
                                   embeddings_regularizer=l1reg)(inp9)
    x9 = tf.keras.layers.Flatten()(x9)
    x9 = tf.keras.layers.Activation(activation)(x9)

    # pod_id id
    inp10 = tf.keras.layers.Input((1,), sparse=False)
    x10 = tf.keras.layers.Embedding(pod_id.max()+1,
                                    kwargs.get('e_pod', emb_size),
                                    embeddings_regularizer=l1reg)(inp10)
    x10 = tf.keras.layers.Flatten()(x10)
    x10 = tf.keras.layers.Activation(activation)(x10)

    # d2v
    inp13 = tf.keras.layers.Input((doc2vec_feats.shape[-1],), sparse=False)
    if kwargs.get('e_doc', emb_size) == -1:
        x13 = inp13
    else:
        x13 = tf.keras.layers.Dense(kwargs.get('e_doc', emb_size),
                                    activation='relu',
                                    kernel_regularizer=l1reg)(inp13)

    # clip
    inp14 = tf.keras.layers.Input((clipmap_train.shape[-1],), sparse=False)
    if kwargs.get('e_clip', emb_size) == -1:
        x14 = inp14
    else:
        x14 = tf.keras.layers.Dense(kwargs.get('e_clip', emb_size),
                                    activation='relu',
                                    kernel_regularizer=l1reg)(inp14)


    # titles
    inp15 = tf.keras.layers.Input((titlemap_train.shape[-1],), sparse=False)
    if kwargs.get('e_title', emb_size*2) == -1:
        x15 = inp15
    else:
        x15 = tf.keras.layers.Dense(kwargs.get('e_title', emb_size*2),
                                    activation='relu',
                                    use_bias=False,
                                    kernel_regularizer=l1reg)(inp15)

    # html svd
    inp16 = tf.keras.layers.Input((htmlmap_train.shape[-1],), sparse=False)
    if kwargs.get('e_html', emb_size) == -1:
        x16 = inp16
    else:
        x16 = tf.keras.layers.Dense(kwargs.get('e_html', emb_size),
                                    activation='relu',
                                    kernel_regularizer=l1reg)(inp16)

    # simweb
    inp17 = tf.keras.layers.Input((simweb_feats.shape[-1],), sparse=False)
    if kwargs.get('e_simweb', emb_size) == -1:
        x17 = inp17
    else:
        x17 = tf.keras.layers.Dense(kwargs.get('e_simweb', emb_size),
                                    activation='relu',
                                    kernel_regularizer=l1reg)(inp17)

    # bigrams
    inp18 = tf.keras.layers.Input((bigrams_feats.shape[-1],), sparse=False)
    if kwargs.get('e_bigram', emb_size) == -1:
        x18 = inp18
    else:
        x18 = tf.keras.layers.Dense(kwargs.get('e_bigram', emb_size),
                                    activation='relu',
                                    kernel_regularizer=l1reg)(inp18)

    x_extra = [
                 x2,
                 x3,
                 x4,
                 x5,
                 x6,
                 x7,
                 x8,
                 x9,
                 x10,
                 x13,
                 x14,
                 x15,
                 x16,
                 x17,
                 x18,
            ]

    x_extra = [xx for xx, remain in zip(x_extra, use_feats) if remain]

    x_sex0 = tf.keras.layers.concatenate([x] + x_extra)
    if kwargs.get('pre_bn', False):
        x_sex0 = tf.keras.layers.BatchNormalization(
            epsilon=1e-5, momentum=0.1)(x_sex0)
    if kwargs.get('pre_dropout', False):
        x_sex0 = tf.keras.layers.Dropout(0.1)(x_sex0)
    x_age0 = x_sex0


    parallel_age = []

    nn_act = kwargs.get('nn_act', 'relu')
    for _ in range(1):
        prev_x_age = [x_age0]
        x_age = x_age0
        for _ in range(kwargs.get('dense_con_num', 2)):
            x2 = tf.keras.layers.Dense(x_age.shape[-1], activation=nn_act,
                                       use_bias=True,
                                       kernel_regularizer=l1reg)(x_age)
            # dense connections
            prev_x_age.append(x2)
            x_age = tf.keras.layers.add(prev_x_age)
            if kwargs.get('bn', False):
                x_age = tf.keras.layers.BatchNormalization(
                    epsilon=1e-5, momentum=0.1)(x_age)
            if kwargs.get('dropout', False)>0:
                x_age = tf.keras.layers.Dropout(kwargs.get('dropout'))(x_age)
        parallel_age.append(x_age)

    if kwargs.get('age_extra_dim', False):
        x_age = tf.keras.layers.concatenate([
            tf.keras.layers.Dense(kwargs.get('age_extra_dim'),
                                  activation=nn_act,
                                  use_bias=True,
                                  kernel_regularizer=l1reg)(x_age0)
        ] + parallel_age)
    else:
        x_age = parallel_age[0]

    if kwargs.get('sex_extra_dim', False):
        x_sex = tf.keras.layers.concatenate([
            tf.keras.layers.Dense(kwargs.get('sex_extra_dim'),
                                  activation=nn_act,
                                  use_bias=True,
                                  kernel_regularizer=l1reg)(x_sex0)
        ] + parallel_age)
    else:
        x_sex = parallel_age[0]

    out1 = tf.keras.layers.Dense(1, activation='sigmoid', use_bias=True, name='sex',
                                 kernel_regularizer=tf.keras.regularizers.l1(l1_reg))(x_sex)

    out2 = tf.keras.layers.Dense(6, activation='softmax', use_bias=True, name='age',
                            kernel_regularizer=tf.keras.regularizers.l1(l1_reg))(x_age)

    inps_extra = [
        inp2,
        inp3,
        inp4,
        inp5,
        inp6,
        inp7,
        inp8,
        inp9,
        inp10,
        inp13,
        inp14,
        inp15,
        inp16,
        inp17,
        inp18
    ]
    inps_extra = [xx for xx, remain in zip(inps_extra, use_feats) if remain]

    model = tf.keras.models.Model([inp] + inps_extra, [out1, out2])

    max_weight = max(sex_weight, age_weight)
    model.compile(loss={'sex':'binary_crossentropy',
                        'age':'categorical_crossentropy'},
                  loss_weights={'sex':sex_weight/max_weight,
                                'age':age_weight/max_weight},
                  optimizer=tf.keras.optimizers.Adam(learning_rate=base_lr,
                                                     clipvalue=kwargs.get('clipvalue', 2.)),
                 )
    return model


def get_scheduler(base_lr=2e-4, factor=1., offset=0.5):
    def scheduler(epoch, lr):
        return base_lr*10**(-epoch*factor+offset)
    return scheduler

## Кешируем фолды кроссвалидации

In [41]:
X_train = csr_matrix((X_tr.data**0.5, X_tr.nonzero()),
                    shape=X_tr.shape,
                    dtype=np.float32)

y_ohe_age = np.zeros((y_all.size, y_all.max()))
y_ohe_age[np.arange(y_all.size), y_all-1] = 1.

CACHE = {}
for k, (train_ind, test_ind) in enumerate(tqdm(folds)):
    # тут выбираем 2-3 случайных номера фолдов
    # чтобы подобранные архитектуры были разнообразными
    if k not in [3, 4]: continue
    train_dat = X_train[train_ind]
    train_y_sex = trg_sex[train_ind]
    train_y_age = y_ohe_age[train_ind]
    val_dat = X_train[test_ind]
    val_y_sex = trg_sex[test_ind]
    val_y_age = y_ohe_age[test_ind]

    train_aux_dat = [train_dat,
                     reg_id[all_mask][train_ind, None],
                     city_id[all_mask][train_ind, None],
                     cpeman_id[all_mask][train_ind, None],
                     cpemodname_id[all_mask][train_ind, None],
                     cpetype_id[all_mask][train_ind, None],
                     price_id[all_mask][train_ind, None]**0.5,
                     cbag_all_train[train_ind],
                     date_id[all_mask][train_ind, None],
                     pod_id[all_mask][train_ind, None],
                     doc2vec_feats_train[train_ind],
                     clipmap_train[train_ind],
                     titlemap_train[train_ind],
                     htmlmap_train[train_ind],
                     simweb_feats_train[train_ind],
                     bigrams_feats_train[train_ind],
                    ]

    val_aux_dat = [val_dat,
                   reg_id[all_mask][test_ind, None],
                   city_id[all_mask][test_ind, None],
                   cpeman_id[all_mask][test_ind, None],
                   cpemodname_id[all_mask][test_ind, None],
                   cpetype_id[all_mask][test_ind, None],
                   price_id[all_mask][test_ind, None]**0.5,
                   cbag_all_train[test_ind],
                   date_id[all_mask][test_ind, None],
                   pod_id[all_mask][test_ind, None],
                   doc2vec_feats_train[test_ind],
                   clipmap_train[test_ind],
                   titlemap_train[test_ind],
                   htmlmap_train[test_ind],
                   simweb_feats_train[test_ind],
                   bigrams_feats_train[test_ind],
                  ]

    with gzip.open('oof_scores_260223/101010/%d.pickle.gz'%(k+1), 'rb') as f:
        oof_scores = pickle.load(f)

    CACHE[k] = [[train_aux_dat, [train_y_sex, train_y_age,
                                 oof_scores['sex'], oof_scores['age']]],
                [val_aux_dat, [val_y_sex, val_y_age]]]

  0%|          | 0/10 [00:00<?, ?it/s]

## Мапка признаков для удобства

In [42]:
feat2idx_map = dict(e_region=0,
                    e_city=1,
                    e_cpeman=2,
                    e_cpemodname=3,
                    e_cpetype=4,
                    e_price=5,
                    e_date=6,
                    e_pod=7,
                    e_cbag=8,
                    e_doc=9,
                    e_clip=10,
                    e_title=11,
                    e_html=12,
                    e_simweb=13,
                    e_bigram=14)

## Функция качества для байесовского поиска

In [43]:
def change_state(model, mode='train'):
    for i, l in enumerate(model.layers):
        model.layers[i].trainable = (mode=='train')
    return model

def get_score(args, times=2):
    BASELINE_SCORE = 47*2
    TIMELIMIT = 60*4

    # delete useless values
    for k, v in args.copy().items():
        if v is None:
            del args[k]
            continue
        if k.startswith('e_') or 'extra' in k:
            args[k] = int(args[k])

    use_feats = [1 for _ in range(len(feat2idx_map))]
    for k, v in feat2idx_map.items():
        use_feats[v] = int(args.get(k, 100) != 1)

    args['use_feats'] = tuple(use_feats)

    epochs = args['epochs'] = int(args['epochs']) #4
    batch_size = args['batch_size'] = int(args['batch_size']) #128+256
    steps_subsample = args['steps_subsample'] = float('%.3f'%args['steps_subsample']) #0.95
    base_lr = args['base_lr'] = args['base_lr'] #2e-4
    factor = args['factor'] = float('%.3f'%args['factor']) #1.
    offset = args['offset'] = float('%.3f'%args['offset']) #0.5
    emb_size = args['emb_size'] = int(args['emb_size']) # 64
    dense_size = args['dense_size'] = int(args['dense_size']) #1024
    sex_weight = args['sex_weight'] = float('%.3f'%args['sex_weight'])
    age_weight = args['age_weight'] = float('%.3f'%args['age_weight'])
    sex_alpha = args['sex_alpha']
    age_alpha = args['age_alpha']
    scheduler = get_scheduler(base_lr, factor, offset)

    AUX_DAT = [
             all_data_sqrt,
             reg_id[:, None],
             city_id[:, None],
             cpeman_id[:, None],
             cpemodname_id[:, None],
             cpetype_id[:, None],
             price_id[:, None]**0.5,
             cbag_all,
             date_id[:, None],
             pod_id[:, None],
             doc2vec_feats,
             clipmap,
             titlemap,
             htmlmap,
             simweb_feats,
             bigrams_feats,
    ]
    AUX_DAT = [xx for xx, remain in zip(AUX_DAT, [1]+use_feats) if remain]

    print(args)

    scores = []
    seconds = []
    f1s = []
    rocaucs = []

    for k, (train_ind, test_ind) in enumerate(tqdm(folds)):
        for _ in range(times):
            if k not in CACHE: continue
            model_nn = get_model_all(**args)
            num_params = sum([w.flatten().size for w in model_nn.get_weights()])
            # ранний выход из-за сложности модели
            if (batch_size > 200 and args['dense_size'] > 2000) or \
               (num_params > 65_000_000):
                return {'loss': 100,
                        'loss_variance': 0.5,
                        'status': STATUS_OK,
                        'params_args': args,
                        'spent_time': seconds,
                        'scores': [f1s, rocaucs, scores]}

            [train_aux_dat, train_y],\
            [val_aux_dat, val_y] = CACHE[k]
            train_y_sex, train_y_age, oof_sex, oof_age = train_y
            val_y_sex, val_y_age = val_y

            train_aux_dat = [xx for xx, remain in zip(train_aux_dat, [1]+use_feats) if remain]
            val_aux_dat = [xx for xx, remain in zip(val_aux_dat, [1]+use_feats) if remain]

            st_time = time.time()
            weights_backup = model_nn.get_weights()

            model_nn.fit(train_aux_dat,
                         [oof_sex*sex_alpha + train_y_sex*(1 - sex_alpha),
                          oof_age*age_alpha + train_y_age*(1 - age_alpha)],
                      batch_size=batch_size,
                      steps_per_epoch=int(steps_subsample*train_ind.size/batch_size),
                      epochs=epochs,
                      callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler)],
                      verbose=False)

            end_time = time.time()
            spent_seconds = end_time - st_time
            if spent_seconds > TIMELIMIT:
                return {'loss': 100,
                        'loss_variance': 0.5,
                        'status': STATUS_OK,
                        'params_args': args,
                        'spent_time': seconds,
                        'scores': [f1s, rocaucs, scores]}

            if args.get('pseudo', False):
                # PSEUDO-labelling
                model_nn = change_state(model_nn, 'test')
                AUX_PREDS = model_nn.predict(AUX_DAT, batch_size=1024)
                model_nn = change_state(model_nn, 'train')
                model_nn.set_weights(weights_backup)
                del weights_backup

                tf.keras.backend.set_value(model_nn.optimizer.lr, args.get('pretrain_lr', base_lr))
                model_nn.fit(AUX_DAT, AUX_PREDS,
                          batch_size=batch_size,
                          epochs=1,
                          verbose=False)

                tf.keras.backend.set_value(model_nn.optimizer.lr, base_lr)
                model_nn.fit(train_aux_dat,
                             [oof_sex*sex_alpha + train_y_sex*(1 - sex_alpha),
                              oof_age*age_alpha + train_y_age*(1 - age_alpha)],
                          batch_size=batch_size,
                          steps_per_epoch=int(steps_subsample*train_ind.size/batch_size),
                          epochs=epochs,
                          callbacks=[tf.keras.callbacks.LearningRateScheduler(scheduler)],
                          verbose=False)

            model_nn = change_state(model_nn, 'test')
            preds = \
            model_nn.predict(val_aux_dat, batch_size=1024)

            end_time = time.time()
            spent_seconds = end_time - st_time

            f1 = \
            f1_score(val_y_age.argmax(axis=1),
                     preds[-1].argmax(axis=1), average='weighted')*100
            rocauc = \
            roc_auc_score(val_y_sex[val_y_sex!=0.5],
                          preds[0].flatten()[val_y_sex!=0.5])*100

            score = f1*2 + (rocauc-50)*2*0
            scores.append(score - BASELINE_SCORE)
            seconds.append(int(spent_seconds))
            f1s.append(f1)
            rocaucs.append(rocauc)

            print(score)
            # ранний выход, если не уложились в бюджет времени
            if spent_seconds > TIMELIMIT*(1+1.5*int(args.get('pseudo', False))):
                return {'loss': 100,
                        'loss_variance': 0.5,
                        'status': STATUS_OK,
                        'params_args': args,
                        'spent_time': seconds,
                        'scores': [f1s, rocaucs, scores]}
            # ранний выход, если конфигурация модели незначительно лучше бейзлайна
            if score < (BASELINE_SCORE + 2.):
                return {'loss': -np.mean(scores),
                        'loss_variance': np.var(scores, ddof=1) if len(scores)>1 else 0.5,
                        'status': STATUS_OK,
                        'params_args': args,
                        'spent_time': seconds,
                        'scores': [f1s, rocaucs, scores]}

    return {'loss': -np.mean(scores),
            'loss_variance': np.var(scores, ddof=1),
            'status': STATUS_OK,
            'params_args': args,
            'spent_time': seconds,
            'scores': [f1s, rocaucs, scores]}


## Пространство поиска

In [ ]:
space = {
    'epochs': hp.quniform('epochs', 2, 4, 1),
    'batch_size': hp.quniform('batch_size', 192, 256+128, 64),
    'steps_subsample': hp.quniform('steps_subsample', 0.9, 1.0, 0.05),
    'base_lr': 10**hp.quniform('base_lr', -5, -2, 0.25),
    'clipvalue': hp.quniform('clipvalue', 0.5, 10, 0.5),
    'factor': hp.quniform('factor', 1, 2, 0.1),
    'offset': hp.quniform('offset', -1., 1., 0.1),
    'emb_size': 64*2**hp.quniform('emb_size', -3, 3, 0.25),
    'dense_size': 1024*2**hp.quniform('dense_size', -3, 1.5, 0.25),
    'sex_weight': hp.quniform('sex_weight', 0.1, 5.0, 0.1),
    'age_weight': hp.quniform('age_weight', 0.1, 5.0, 0.1),
    'dense_con_num': hp.choice('dense_con_num', [None, 1, 3, 4]),
    'pre_bn': hp.choice('pre_bn', [None, True]),
    'bn': hp.choice('bn', [None, True]),
    'pre_dropout': hp.choice('pre_dropout', [None, True]),
    'dropout': hp.choice('dropout', [None, 0.05, 0.1, 0.2]),
    'nn_act': hp.choice('nn_act', [None, 'elu', 'tanh', 'sigmoid']),

    'pretrain_lr': 10**hp.quniform('pretrain_lr', -5, -2, 0.25),
    'pseudo': hp.choice('pseudo', [False]),

    'sex_alpha': hp.quniform('sex_alpha', 0.0, 1.0, 0.05),
    'age_alpha': hp.quniform('age_alpha', 0.0, 1.0, 0.05),

    'l1_reg': 10**hp.quniform('l1_reg', -8, -4, 0.25),

    'e_region': hp.choice('b1',
                       [None,
                        1,
                        64*2**hp.quniform('region', -3, 3, 0.25)]),
    'e_city': hp.choice('b2',
                     [None,
                      1,
                      64*2**hp.quniform('city', -3, 3, 0.25)]),
    'e_cpeman': hp.choice('b3',
                       [None,
                        1,
                        64*2**hp.quniform('cpeman', -3, 3, 0.25)]),
    'e_cpemodname': hp.choice('b4',
                           [None,
                            1,
                            64*2**hp.quniform('cpemodname', -3, 3, 0.25)]),
    'e_cpetype': hp.choice('b5',
                        [None,
                         1,
                         64*2**hp.quniform('cpetype', -3, 3, 0.25)]),
    'e_price': hp.choice('b6',
                      [None,
                       1,
                       64*2**hp.quniform('price', -3, 3, 0.25)]),
    'e_date': hp.choice('b7',
                     [None,
                      1,
                      64*2**hp.quniform('date', -3, 3, 0.25)]),
    'e_pod': hp.choice('b8',
                    [None,
                     1,
                     64*2**hp.quniform('pod', -3, 3, 0.25)]),
    'e_cbag': hp.choice('b10',
                     [None,
                      1,
                      128*2**hp.quniform('cbag', -3, 3, 0.25)]),
    'e_title': hp.choice('b9',
                      [None,
                       1,
                       128*2**hp.quniform('title', -3, 3, 0.25)]),
    'e_doc': hp.choice('b11',
                       [None,
                        -1,
                        1,
                        64*2**hp.quniform('doc', -3, 3, 0.25)]),
    'e_clip': hp.choice('b12',
                       [None,
                        -1,
                        1,
                        64*2**hp.quniform('clip', -3, 3, 0.25)]),
    'e_html': hp.choice('b13',
                       [None,
                        -1,
                        1,
                        64*2**hp.quniform('html', -3, 3, 0.25)]),
    'e_simweb': hp.choice('b14',
                       [None,
                        -1,
                        1,
                        64*2**hp.quniform('simweb', -3, 3, 0.25)]),
    'e_bigram': hp.choice('b15',
                          [None,
                           1,
                           128*2**hp.quniform('bigram', -4, 2, 0.25)]),

    'age_extra_dim': hp.choice('use_age_extra',
                              [None,
                               64*2**hp.quniform('age_extra_dim', -5, 5, 0.5)]),
    'sex_extra_dim': hp.choice('use_sex_extra',
                              [None,
                               64*2**hp.quniform('sex_extra_dim', -5, 5, 0.5)]),

    'use_emb_act': hp.choice('use_emb_act', [None, 'relu', 'elu', 'tanh'])
}

## Байесовский поиск с чекпоинтами

In [45]:
trials = Trials()
trials_file = 'trials280323age_%s'%(''.join(map(str, sorted(list(CACHE.keys())))))+\
              '_%d.pickle.gz'

n_checkpoints = 2

trials_length = 0
for i in range(n_checkpoints):
    if os.path.exists(trials_file%(i+1)):
        with gzip.open(trials_file%(i+1), 'rb') as f:
            tmp = pickle.load(f)
            if len(tmp.trials) > trials_length:
                trials_length = len(tmp.trials)
                trials = tmp
        print('Loaded %s'%(trials_file%(i+1)))


suggester = partial(tpe.suggest, n_startup_jobs=50, n_EI_candidates=25, gamma=0.25)
for k_iter in range(1000):
    print(len(trials.trials))
    try:
        best = fmin(get_score, space, suggester,
                    max_evals=len(trials.trials) + 1,
                    rstate=None, trials=trials)
    except KeyboardInterrupt:
        break
    except Exception as e:
        print(e)
        continue
    with gzip.open(trials_file%(k_iter%n_checkpoints+1), 'wb') as f:
        pickle.dump(trials, f, protocol=pickle.HIGHEST_PROTOCOL)

    if k_iter % 3 == 0:
        clear_output()

277
{'age_alpha': 0.55, 'age_extra_dim': 45, 'age_weight': 4.4, 'base_lr': 0.0031622776601683794, 'batch_size': 192, 'bn': True, 'clipvalue': 1.0, 'dense_size': 861, 'dropout': 0.05, 'e_bigram': 128, 'e_cbag': 256, 'e_city': 128, 'e_cpeman': 19, 'e_cpemodname': 1, 'e_cpetype': 64, 'e_doc': -1, 'e_html': -1, 'e_region': 26, 'emb_size': 90, 'epochs': 4, 'factor': 1.2, 'l1_reg': 3.162277660168379e-07, 'nn_act': 'sigmoid', 'offset': -0.3, 'pre_dropout': True, 'pretrain_lr': 0.0031622776601683794, 'pseudo': False, 'sex_alpha': 0.65, 'sex_weight': 1.3, 'steps_subsample': 1.0, 'use_emb_act': 'tanh', 'use_feats': (1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)}
100%|█████████▉| 277/278 [00:00<?, ?trial/s, best loss=?]

  0%|          | 0/10 [00:00<?, ?it/s]

/data/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_726/dense_8730/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_726/dense_8730/embedding_lookup_sparse/Reshape:0", shape=(None, 861), dtype=float32), dense_shape=Tensor("gradient_tape/model_726/dense_8730/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)

/data/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_726/dense_8734/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_726/dense_8734/embedding_lookup_sparse/Res

96.77138968597038                                        
100%|█████████▉| 277/278 [03:29<?, ?trial/s, best loss=?]

/data/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_727/dense_8740/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_727/dense_8740/embedding_lookup_sparse/Reshape:0", shape=(None, 861), dtype=float32), dense_shape=Tensor("gradient_tape/model_727/dense_8740/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)

/data/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_727/dense_8744/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_727/dense_8744/embedding_lookup_sparse/Res

96.43578456472916                                        
100%|█████████▉| 277/278 [07:07<?, ?trial/s, best loss=?]

/data/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_728/dense_8750/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_728/dense_8750/embedding_lookup_sparse/Reshape:0", shape=(None, 861), dtype=float32), dense_shape=Tensor("gradient_tape/model_728/dense_8750/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)

/data/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_728/dense_8754/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_728/dense_8754/embedding_lookup_sparse/Res

97.00203960897477                                        
100%|█████████▉| 277/278 [10:48<?, ?trial/s, best loss=?]

/data/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_729/dense_8760/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_729/dense_8760/embedding_lookup_sparse/Reshape:0", shape=(None, 861), dtype=float32), dense_shape=Tensor("gradient_tape/model_729/dense_8760/embedding_lookup_sparse/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)

/data/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:449: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model_729/dense_8764/embedding_lookup_sparse/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model_729/dense_8764/embedding_lookup_sparse/Res

100%|█████████▉| 277/278 [13:44<?, ?trial/s, best loss=?]


## Вывод 3 лучших конфигураций моделей

In [46]:
# fold%3 <=> 3 4
sorted(trials.trials, key=lambda x: x.get('result', {'loss':100}).get('loss', 100))[:3]

[{'state': 2,
  'tid': 262,
  'spec': None,
  'result': {'loss': -3.1522602500615413,
   'loss_variance': 0.04850040593790047,
   'status': 'ok',
   'params_args': {'age_alpha': 0.7000000000000001,
    'age_extra_dim': 32,
    'age_weight': 4.2,
    'base_lr': 0.0017782794100389228,
    'batch_size': 256,
    'bn': True,
    'clipvalue': 1.0,
    'dense_size': 608,
    'dropout': 0.05,
    'e_bigram': 107,
    'e_cbag': 181,
    'e_city': 90,
    'e_cpeman': 38,
    'e_cpemodname': 1,
    'e_cpetype': 90,
    'e_doc': -1,
    'e_html': -1,
    'e_pod': 64,
    'e_region': 90,
    'emb_size': 76,
    'epochs': 4,
    'factor': 1.0,
    'l1_reg': 3.162277660168379e-07,
    'nn_act': 'elu',
    'offset': 0.2,
    'pre_dropout': True,
    'pretrain_lr': 0.0001,
    'pseudo': False,
    'sex_alpha': 0.55,
    'sex_weight': 2.2,
    'steps_subsample': 1.0,
    'use_emb_act': 'tanh',
    'use_feats': (1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)},
   'spent_time': [167, 166, 165, 176],
   'sc

Аналогично подбираются гиперпараметры для максимизации пола и для разных сочетаний поднаборов фолдов